In [98]:
import os
from cryptography.hazmat.primitives.ciphers import (Cipher, algorithms, modes)
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes, hmac
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import dsa
from cryptography.hazmat.primitives.asymmetric import ec

def nounceGenerator(tam):
    return os.urandom(tam)

def cifragem(plaintext):
    iv = nounceGenerator(12)
    password = b'chave secreta'
    nounce = nounceGenerator(16)
    
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=nounce,
        iterations=100000,
        backend=default_backend()
    )
    key = kdf.derive(password) 
    
    encryptor = Cipher(
        algorithms.AES(key),
        modes.GCM(iv),
        backend=default_backend()
    ).encryptor()
    
    ciphertext = encryptor.update(plaintext) + encryptor.finalize()
    
    private_key = dsa.generate_private_key(key_size=1024,)
    sign = private_key.sign(ciphertext, hashes.SHA256())
    
    return (ciphertext, iv, encryptor.tag, nounce, sign, private_key.public_key())


def decifragem(ciphertext, iv, tag, nounce, sign, public_key):
    password = b'chave secreta'
    
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=nounce,
        iterations=100000,
        backend=default_backend()
    )
    key = kdf.derive(password) 
    
    
    public_key.verify(
        sign,
        ciphertext,
        hashes.SHA256()
    )
    
    
    
    decryptor = Cipher(
        algorithms.AES(key),
        modes.GCM(iv, tag),
        backend=default_backend()
    ).decryptor()
    
    return decryptor.update(ciphertext) + decryptor.finalize()

In [99]:
(a, b, c, d, e, f) = cifragem(b'boas maltinha')
print(a, b, c, d, e, f)
res = decifragem(a, b, c, d, e, f)
print(res)

b'\t\xbc\xab\x8b\xb8k\x95\xd9\x90\xe1\xdf\x94\x81' b'\xcc\x0f\xf1L\xc2\x08?]\xe9\x9aT\xae' b'\xca\xc4\x19\x1f\xe0\x84\xc3!(%\xdf:\xb4\xc8A\x10' b'\x0ch*\xb1?7\x8b>n\xe3\xa5\xd2Y8j\xf9' b'0-\x02\x15\x00\x99N\xc5\xe8\xfe;\xec\x9f\xb2\x1f\x15\xeb\xcfF\xdc\xa6i\xdau$\x02\x14z\x8fR\xdd\x89\xaa\xcb\xad#\xb5\x0fJ\x04\x8f\x83.\xb1.\xcc6' <cryptography.hazmat.backends.openssl.dsa._DSAPublicKey object at 0x00000275865B4910>
b'boas maltinha'


In [100]:
def cifragem2(plaintext):
    iv = nounceGenerator(12)
    password = b'chave secreta'
    nounce = nounceGenerator(16)
    
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=nounce,
        iterations=100000,
        backend=default_backend()
    )
    key = kdf.derive(password) 
    
    encryptor = Cipher(
        algorithms.AES(key),
        modes.GCM(iv),
        backend=default_backend()
    ).encryptor()
    
    ciphertext = encryptor.update(plaintext) + encryptor.finalize()
    
    private_key = ec.generate_private_key(
    ec.SECP384R1()
    )
    sign = private_key.sign(
    ciphertext,
    ec.ECDSA(hashes.SHA256())
    )
    
    return (ciphertext, iv, encryptor.tag, nounce, sign, private_key.public_key())


def decifragem2(ciphertext, iv, tag, nounce, sign, public_key):
    password = b'chave secreta'
    
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=nounce,
        iterations=100000,
        backend=default_backend()
    )
    key = kdf.derive(password) 
    
    
    public_key.verify(sign, ciphertext, ec.ECDSA(hashes.SHA256()))
    
    
    
    decryptor = Cipher(
        algorithms.AES(key),
        modes.GCM(iv, tag),
        backend=default_backend()
    ).decryptor()
    
    return decryptor.update(ciphertext) + decryptor.finalize()

In [101]:
(a, b, c, d, e, f) = cifragem2(b'boas maltinha')
print(a, b, c, d, e, f)
res = decifragem2(a, b, c, d, e, f)
print(res)

b'3\xd7d/\xac\xe9p]M\xe5\xdc\xa4\xec' b"\xa4\xb1\x87A'P:\x9aG\x8dB\xd3" b'\x81(;\xb2\xaaC\x9c\xff>\xaa\x82\x1a\x0e\xe6\xf9\xdd' b'~\x92\xac\xc2\xe9\xf5\xce\xfd\x95\xd6\xf3,\n\xce\x058' b'0f\x021\x00\x8d\xa9\x89\x86f\xeff\x0b\xc0 G\xd7\xf9wo\x88<pK\x19\xb5\xccy\x9c\xd7\x81\x14+\x80l`\xf0T\xa9\x02\xc0\xfd\x13\x00\x03\x01\x19u\xba\xdeY\x9d\x00\x021\x00\xaf\x13?\xc4~"U$ Ca(\x1f(\xbc\xbbW\rN\xa83\xc87E{.\xef=g\x14q\xf9\xf7\x13\x95N\x91]\x85\x18@\x1e\xfc\x8cv\x90\x1ax' <cryptography.hazmat.backends.openssl.ec._EllipticCurvePublicKey object at 0x00000275865D5CD0>
b'boas maltinha'
